In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-25 05:02:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-05-25 05:02:11 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData").config("spark.driver.extraClassPath=/home/hadoop/postgresql-42.2.16.jar","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame (extract)

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [5]:
# Count the number of records (rows) in the dataset.
df.count()

4057147

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
# transform review_date into Date field
from pyspark.sql.functions import to_date

# create review_id_df
review_id_df2 = df.select(['review_id','customer_id','product_id','product_parent', to_date('review_date', 'yyyy-mm-dd')]).drop_duplicates(['review_id'])
review_id_df2.show()

+--------------+-----------+----------+--------------+--------------------------------+
|     review_id|customer_id|product_id|product_parent|to_date(review_date, yyyy-mm-dd)|
+--------------+-----------+----------+--------------+--------------------------------+
|R10001EP8WFB6B|   34517685|B007FUI0DC|      16851403|                      2013-01-10|
|R1000DTU1XTO4B|   37082663|B00MFDPI0M|     807463691|                      2014-01-13|
|R1000FJ09F5T3S|   14779632|B0099JKR6U|     886803651|                      2013-01-04|
|R1000KU66CGPXT|   44376875|B00X8UKOUK|     666093513|                      2015-01-08|
|R1000WFE9T36XC|   47251216|B002WQ3VLA|     462748064|                      2014-01-13|
|R10016HHTLINTP|   36803385|B00RG11S5C|     531931687|                      2015-01-26|
|R1001AFAN4B61I|    4337929|B00U0AA2RQ|     922806508|                      2015-01-21|
|R1001EWPHYEFT4|   30608149|B000YMM3CC|     653409458|                      2012-01-20|
|R1001L69YMAP94|   25755433|B007

In [11]:
review_id_df2.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- to_date(review_date, yyyy-mm-dd): date (nullable = true)



In [12]:
customers_df2 = df.groupby('customer_id').agg({'customer_id':'count'})
customers_df2 = customers_df.withColumnRenamed('count(customer_id','customer_count')
customers_df2.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   36427874|                 1|
|   17687096|                10|
|   43798350|                 4|
|   49387236|                 1|
|   24321836|                 2|
|   42543176|                 1|
|   16298288|                 4|
|   12333316|                 1|
|   25790304|                 1|
|   13786319|                 1|
|   24796052|                 1|
|   53049917|                 4|
|   30478898|                 1|
|   20829560|                 3|
|   29699080|                 3|
|    8681732|                 3|
|   34220092|                 2|
|   13727857|                14|
|   13480593|                 3|
|   49935991|                 1|
+-----------+------------------+
only showing top 20 rows



In [13]:
customers_df2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- count(customer_id): long (nullable = false)



In [14]:
# Load into AWS RDS instance
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cpklvwgn9puo.us-east-1.rds.amazonaws.com:5432/video_review_db"
config = {"user":"postgres", "password": "postgres","driver":"org.postgresql.Driver"}

In [15]:
review_id_df2.write.jdbc(url=jdbc_url, table='review_id_df2', mode=mode, properties=config)

Py4JJavaError: ignored

In [16]:
customers_df2.write.jdbc(url=jdbc_url, table='customers_df2', mode=mode, properties=config)

Py4JJavaError: ignored